# ✨Prodigy 

[Prodigy Documentation](https://spacy.apjan.co/docs/)

Exercise I: Basic NER with pre-trained spaCy models [source](https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da)

In [ ]:
import os 
import pickle
import spacy
from spacy import displacy
from collections import Counter

import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
doc = nlp(
    "Hosted by Utrecht University, the 2019 iteration of the Digital Humanities (DH) conference, the annual international conference of the Alliance of Digital Humanities Organizations, will take place in the medieval city of Utrecht, one of the oldest cities in the Netherlands. The city’s rapid modernization and growth has inspired the conference’s guiding theme, complexity."
)
displacy.render(doc, style="ent")


## Problem
This works very well for many 20th and 21st century texts.  But what about 17th century English?

In [ ]:
doc = nlp(
    "ITEM because that the kings most deare Uncle, the king of Denmarke, Norway & Sweveland, as the same our soveraigne Lord the king of his intimation hath understood, considering the manifold & great losses, perils, hurts and damage which have late happened aswell to him and his, as to other foraines and strangers, and also friends and speciall subjects of our said soveraigne Lord the king of his Realme of England, by ye going in, entring & passage of such forain & strange persons into his realme of Norwey & other dominions, streits, territories, jurisdictions & places subdued and subject to him, specially into his Iles of Fynmarke, and elswhere, aswell in their persons as their things and goods"
)
displacy.render(doc, style="ent")


## Introduction to ✨Prodigy

Prodigy is an annotation tool for active machine teaching.  Prodigy makes it possible to quickly experiment and improve models with relatively little data, time and typically only one annotator.  Prodigy will sort a model's results by certainty, and ask for human input where the results are most ambiguous. Prodigy stores these annotations and uses them to update the model.  Machine teaching can be used to: 

- improve an existing model for a specific context or set of documents
- add a new entity or category to a text model 
- custom image categorization and object recognition

Prodigy does require a license to use. Researchers at degree-granting academic institutions can request a free research license [here](https://prodi.gy/forms/research-license)

In this example, our goal is to teach an existing English-language model to identify early modern place names.

There are several approaches that we could take to this problem.  Different approaches can lend better or worse result and experimentation is an essential part of any machine learning project. 

## How can we teach a statistical language model that Sweveland is a place?



**Manual annotation**

Some researchers may prefer to add seed annotations manually.  Using Prodigy, you can add seed terms by hand.   
`prodigy ner.manual historic_places en_core_web_sm principal_navigations.txt --label PLACE`
For certain personalities, myself included, this is actually kind of fun and a good way to think about the text and the goals of your experiment.   

For the current problem, this is not my suggested method.  Place names are very distinct and hard to identify from context.  If the model knew that York is a place, how could it learn that New York is also a place? By contrast, titles typically occur before proper names.  It could learn that both Professor and Doctor are titles without knowing that those specific terms are used as titles.  In our current case, it's best to give the model as many examples of place names as possible.  Note that with spaCy, we are working with place names as [patterns](https://spacy.apjan.co/docs/#match-patterns) so the model can learn that place names often have several parts.  

**Seed term patterns** 

We're going to give the model as many examples of historic places names as we can to get the learning process started. To do this, I have chosen to mine a text that has already been marked up in TEI with early modern place names and is available from the Perseus Project.  We're going to use Richard Hakluyt's [*The Principal Navigations, Voyages, Traffiques, and Discoveries of the English Nation* (1599)](http://www.perseus.tufts.edu/hopper/text?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D1).  I have chosen an English-language example because we'll be annotating the text in this workshop and English is a working language for DH2019.  However, you can work with texts in [any language](https://spacy.io/usage/adding-languages) even those that do not have an existing spaCy language model ([colonial Zapotec](https://ticha.haverford.edu/) for example!).    



**New category or existing?** 

We will need to choose whether we want to add a new entity to the model (let's call it "PLACE") or improve the existing "GPE" entity for our historic places.  Either one works.  If we improve an existing entity, we retain the existing training and examples. A new entity makes it easier to distinguish between what we have taught the model and previous training.  This can also address potential bias in previous training data.  For that reason, we'll add "PLACE" as a new category.     

### Download the TEI files from Persius 
- We're going to extract a list of all the place names from the text to create a patterns JSONL file.
- We'll also extract the raw text to create a set of training documents. 

We are going to download the table of contents and create a list of the 937 segments of the document. We will then get each page, remove the place names (`<name type="place">Utrect</name>`) and add them to a places list.

In [ ]:
from urllib.request import urlopen
from lxml import etree
spec = {"tei":"http://www.tei-c.org/ns/1.0"}

def tei_loader(url):
    tei = urlopen(url).read()
    return etree.XML(tei)

table_of_contents_url = "http://www.perseus.tufts.edu/hopper/xmltoc?doc=Perseus%3Atext%3A1999.03.0070%3Anarrative%3D1"
table_of_contents_xml = tei_loader(table_of_contents_url)

if not os.path.exists('refs.pickle'):
    chunks = table_of_contents_xml.xpath("//chunk[@ref]")
    refs = [chunk.get('ref') for chunk in chunks] 
    # an example ref 'Perseus%3Atext%3A1999.03.0070%3Anarrative%3D6'


    places = []

    for ref in refs:

        url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + ref
        try:
            tei = tei_loader(url)

            #get all <name type='place'> tags
            for place in tei.findall(".//name[@type='place']", namespaces=spec):
                places.append(place.text.replace('\n',''))
        except Exception as e:
            print(e)
            
    pickle.dump(places, open('places.pickle', 'wb'))
    pickle.dump(refs, open('refs.pickle', 'wb'))

else:
    places = pickle.load(open('places.pickle', 'rb'))
    refs = pickle.load(open('refs.pickle', 'rb'))
    print('pickles loaded')


In [ ]:
print('number of documents: ', len(refs))
print('number of places found: ', len(set(places)))
places[9]

### Create a patterns.jsonl file with seed terms.  
These terms provide examples that the model can use to learn the new category. It is good to use as many terms as are practical; preferably 100-200. Further examples of patterns files can be found [here](https://github.com/explosion/prodigy-recipes/tree/master/example-patterns) and there is a very helpful tool for creating patterns that are relevant to your projects and texts [here](https://explosion.ai/demos/matcher)


In [ ]:
import json
new_label = 'PLACE'

if not os.path.exists('patterns.jsonl'):
    with open('patterns.jsonl','w') as f:
        for place in set(places):   # A set is used here to remove duplicate place names
            
            row = {}
                
            row['label'] = new_label
            row['pattern'] = []
            for token in place.split():
                pattern = {}
                pattern['lower'] = token.lower()
                row['pattern'].append(pattern)
        
            f.write(str(json.dumps(row) +'\n')) 
            
        '''Polite intervention:  
        Sweveland is not in our list of historic and beautiful places, but is essential to the narrative of this notebook.
        To correct this error, the line below has been added, with appologies to our colleagues from Sweveland'''
        row = {"label": "PLACE", "pattern": [{"lower": "sweveland"}]}
        f.write(json.dumps(row))

In [ ]:
with open('patterns.jsonl','r') as f:
    print(f.read()[:185])

In [ ]:
#Now to extract the full text
percent_of_texts_to_process = 0.05 
'''We are only using a tiny portion of the text for annotation.  
Unlike many machine learning tasks, in this case, more text data for annotations does not necessarily
improve learning. More text will dramatically increase Prodigy's memory usage, so keep that in mind.'''

end_index = int(len(refs) * percent_of_texts_to_process)

if os.path.exists("principal_navigations.txt"):
    print(
        "The file already exists. This process takes several minutes. To re-run, change process_anew to True. You can also adjust the percentage of the corpus to be gathered."
    )
    
process_anew = False
if process_anew:
    txts = []
    for ref in refs[:end_index]:

        url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + ref
        try:
            tei = tei_loader(url)

            new_txt = []
            for body in tei.iter('body'):
                new_txt.append(''.join(body.itertext()).strip().replace('\n',''))
                txts.append(''.join(new_txt))

        except Exception as e:
            print(e)
            

    full_text = [
        txt.replace("        ", " ").replace("   ", " ").replace("  ", " ")
        for txt in txts
    ]
    
    with open('principal_navigations.txt','w') as f:
        f.write(str(full_text))

In [ ]:
with open('principal_navigations.txt','r') as f: 
    print(f.read()[:600])
    

## With patterns and text files created, we can now work with ✨Prodigy!

The `dataset` command will create a database table to save your annotations. The default is sqlite, but you can connect to [MySQL or postgres](https://spacy.apjan.co/docs/#database-setup)

In [ ]:
!prodigy dataset historic_places "A dataset for British historic places" --author Andy

If you'd like to delete a dataset:   

`prodigy drop historic_places`



## A. Plaintext to TEI. 


To start the annotation application for a named entity recognition task, we use the `ner.teach` recipe. Similar [built-in recipes](https://spacy.apjan.co/docs/#built-in-recipes) are available for: 
- text categorization (textcat.teach) 
- part of speech tagging (pos.teach)  
- vectors & terminology (terms.teach)
- computer vision (image.manual)

In the command below, we use the `ner.teach` recipe to annotate the `historic_places` dataset, using the `en_core_web_sm` model to add the new entity `PLACE` after loading the patterns in `patterns.jsonl`.  

**Please be patient, this next step can take a few minutes to load**  
Click on the stop button to intterupt the kernel when you're done. 

In [ ]:
!prodigy ner.manual historic_places en_core_web_sm principal_navigations.txt --label PLACE

In [ ]:
!prodigy ner.teach historic_places en_core_web_sm principal_navigations.txt --label PLACE --patterns patterns.jsonl

Once you have completed adding annotations, the next step is to train the model.  

In the command below, we use `ner.batch-train` to use the annotations in the `historic_places` dataset to train the `en_core_web_sm` model on the new entity `PLACE`.  We then save the updated model as `new_model`.

In [ ]:
!prodigy ner.batch-train historic_places en_core_web_sm --label PLACE --output new_model 

If you think the model would benefit from more training you can run this process again to load and update `new_model`.  You can also add the `--n-iter` argument to specify the number of iterations.

In [ ]:
!prodigy ner.batch-train historic_places new_model --output new_model --label PLACE --n-iter 120

Would our model improve with more data?

In [ ]:
!prodigy ner.train-curve historic_places new_model --label PLACE --n-iter 10 --eval-split 0.2 --dropout 0.2  --n-samples 4 

To load and see the results of our new model 

In [ ]:
import spacy

nlp = spacy.load("new_model")
doc = nlp(
    "ITEM because that the kings most deare Uncle, the king of Denmarke, Norway & Sweveland, as the same our soveraigne Lord the king of his intimation hath understood, considering the manifold & great losses, perils, hurts and damage which have late happened aswell to him and his, as to other foraines and strangers, and also friends and speciall subjects of our said soveraigne Lord the king of his Realme of England, by ye going in, entring & passage of such forain & strange persons into his realme of Norwey & other dominions, streits, territories, jurisdictions & places subdued and subject to him, specially into his Iles of Fynmarke, and elswhere, aswell in their persons as their things and goods"
)
displacy.render(doc, style="ent")

That's not too bad.  What about a text that the model has never seen before? Let's try Çelebi Evliya's [Narrative of travels in Europe, Asia, and Africa](https://archive.org/details/narrativeoftrave01evli/page/n4)

In [ ]:
import spacy

nlp = spacy.load("new_model")
doc = nlp(
    """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634).  
"""
)
counter = 0
for ent in doc.ents:
    if ent.text in places:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
        counter += 1

print(f"{counter} of the place ids were in the training data")
displacy.render(doc, style="ent")

Mustafâ Pâshâ is obviously a person, but the model has done as passable job. Let's use our model to automatically identify historical place names and produce a markedup TEI document. 

In [ ]:
text = """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634)."""  

import spacy
nlp = spacy.load('new_model')
doc = nlp(text)
text_list = [i.text for i in doc]

for token in doc:
    if token.ent_type_ == 'PLACE': 
        text_list[token.i] = '<name type="place">' + text_list[token.i] + '</name>'

punct = ['.',"'",',',')',':',';']

text=''
for i, token in enumerate(text_list):
    try:
        if text_list[i+1] in punct:
            text += token

        else: 
            text += token + ' '
        
    except IndexError:
        pass
   
    
text.replace('\n','').replace('( ','(')

In [ ]:
# save text as tei 
filename = 'my_tei.xml'
language = 'en'

tei_string = f"""
<TEI.2>
  <text lang="{language}">
    <body>
        <p>
            {text}
        </p>
    </body>
  </text>
</TEI.2>
"""
doc = etree.fromstring(tei_string)
tree = etree.ElementTree(doc)
tree.write(f'{filename}', pretty_print=True, xml_declaration=False,   encoding="utf-8")


In [ ]:
with open('my_tei.xml','r') as f:
    print(f.read())

### B. TEI to TEI


In [ ]:
!prodigy dataset historic_places_xml "A dataset for British historic places with TEI input" --author Andy

In [ ]:
#save xml file to disk
with open('my_tei.txt','w') as f:
    url = 'http://www.perseus.tufts.edu/hopper/xmlchunk?doc=' + refs[2]
    print(url)
    tei = urlopen(url).read()
    tei = etree.XML(tei)
    text = etree.tostring(tei)
    f.write(str(text))

with open('my_tei.txt','r') as fr:
    print(fr.read()[:100])

In [ ]:
!prodigy ner.teach historic_places_xml en_core_web_sm my_tei.txt --label PLACE --patterns patterns.jsonl

## Prodigy as a manual annotation tool 
text > tei manual markup

In [ ]:
text = """The army marched from Konia to Kaiseria (Caesarea), and thence to Sivas, where the feast of the Korbân (sacrifice) was celebrated. Here Mustafâ Pâshâ, the emperor's favourite, was promoted to the rank of second vezir, and called into the divân. The army then continued its march to Erzerum. Besides tiie guns provided by the commander-in-chief, there were forty large guns dragged by two thousand pairs of buftaloes. The army entered the castle of Kazmaghan, and halted under the walls of Eriviin in the year 1044 (1634)."""  
with open('text2tei.txt','w') as f:
    f.write(text)

In [ ]:
!prodigy dataset text2tei "A dataset for British historic places with text input" --author Andy

In [ ]:
!prodigy ner.manual text2tei en_core_web_sm text2tei.txt

In [ ]:
!prodigy db-out text2tei .

In [ ]:
with open('text2tei.jsonl','r') as f:
    jsonl = json.loads(f.read())
    print(jsonl['spans'])

In [ ]:
import json 

def jsonl_to_xml(jsonl):    
    jsonl = json.loads(jsonl)
    text = jsonl['text']
    offset = 0
    
    for span in jsonl['spans']:

        new_text = f"<{span['label']}>" + text[span["start"] +offset : span["end"] + offset] +  f"</{span['label']}>" 
        text = text[:span["start"] + offset] + new_text + text[span["end"] + offset:]
        offset += len(new_text) - (span["end"] - span["start"])
    
    return text
        

with open('text2tei.jsonl','r') as f:
    jsonl = f.read()
    xml = jsonl_to_xml(jsonl)
    print(xml)

In [ ]:
# save text as tei 
filename = 'annotated_tei.xml'
language = 'en'

tei_string = f"""
<TEI.2>
  <text lang="{language}">
    <body>
        <p>
            {xml}
        </p>
    </body>
  </text>
</TEI.2>
"""
doc = etree.fromstring(tei_string)
tree = etree.ElementTree(doc)
tree.write(f'{filename}', pretty_print=True, xml_declaration=False,   encoding="utf-8")

with open(filename,'r') as f:
    print(f.read())

## Thank you for your attention!  How could you use spaCy and Prodigy in your own projects?  